# 1. selenium / kaggle 

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import os
import shutil
import time
import zipfile

In [2]:
# clean run
try:
    os.remove(os.path.join(os.curdir, 'all.zip'))
    shutil.rmtree(os.path.join(os.curdir, 'titanic'))
except:
    pass

In [3]:
from selenium.webdriver import Chrome

driver_path="../chromedriver_win32/chromedriver.exe"

download_path = os.path.abspath(os.curdir)
driver_options = webdriver.ChromeOptions() 
driver_options.add_experimental_option('prefs', {'download.default_directory' : download_path})

driver = Chrome(executable_path=driver_path, options=driver_options)
driver.get('https://www.kaggle.com/')

In [4]:
a_competitions = driver.find_element_by_xpath('//a[@href="/competitions"]')
a_competitions.click()

In [5]:
a_titanic = driver.find_element_by_xpath('//div[contains(@class,"competition-info")]/div/a[@title="Titanic: Machine Learning from Disaster"]')
a_titanic.click()

In [6]:
def click_data():
    a_data = driver.find_element_by_xpath('//a[@id="pageheader-nav-item--data"]')
    a_data.click()
    
click_data()

In [ ]:
input_username = driver.find_element_by_xpath('//input[@id="username-input-text"]')
input_username.clear()
input_username.send_keys('<username>')

In [ ]:
input_password = driver.find_element_by_xpath('//input[@id="password-input-text"]')
input_password.clear()
input_password.send_keys('<Passw0rd>')
input_password.submit()

In [9]:
click_data()

In [10]:
a_download_all = driver.find_element_by_xpath('//a[contains(@class,"DownloadAllButton")and(@href)]')
a_download_all.click()

In [11]:
time.sleep(5)
driver.quit()

In [12]:
if 'all.zip' in os.listdir(download_path):
    all_zip = zipfile.ZipFile(os.path.join(download_path,'all.zip'), 'r')
    all_zip.extractall(os.path.join(os.path.abspath(os.curdir), 'titanic'))
    all_zip.close()
os.listdir(download_path) # all.zip, titanic/

['.ipynb_checkpoints',
 'all.zip',
 'catboost_info',
 'Homework_11.ipynb',
 'titanic']

# 2. catboost / titanic 

In [13]:
import pandas as pd
from catboost import CatBoostClassifier, Pool

In [14]:
train = pd.read_csv('titanic/train.csv')
test = pd.read_csv('titanic/test.csv')

_Первая буква в названии каюты означает номер палубы (https://www.encyclopedia-titanica.org/). Я предположил, что это более значимый признак сам по себе, поэтому решил выделить его отдельно._

In [15]:
def prepare_dataset(df):
    df[['Cabin', 'Embarked']] = df[['Cabin', 'Embarked']].fillna('Unknown')
    df['Deck'] = df['Cabin'].apply(lambda s: s[0])
    return df

In [16]:
train = prepare_dataset(train)

In [17]:
print( train[['Cabin', 'Embarked']].isnull().sum() )

Cabin       0
Embarked    0
dtype: int64


In [18]:
print( train['Deck'].value_counts() )

U    687
C     59
B     47
D     33
E     32
A     15
F     13
G      4
T      1
Name: Deck, dtype: int64


In [19]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Deck
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,Unknown,S,U
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,Unknown,S,U
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,C
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,Unknown,S,U


In [20]:
X_train = train.drop(['Survived', 'PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
y_train =  train[['Survived']]

In [21]:
X_train.dtypes.to_frame('dtype').reset_index()

,index,dtype
0,Pclass,int64
1,Sex,object
2,Age,float64
3,SibSp,int64
4,Parch,int64
5,Fare,float64
6,Embarked,object
7,Deck,object


In [22]:
model = CatBoostClassifier(iterations=800, learning_rate=0.01, depth=5, loss_function='MultiClass', verbose=False)

In [23]:
model.fit(Pool(X_train, y_train, cat_features=[1, 6, 7]));

In [24]:
pd.DataFrame(index=X_train.columns, data=model.feature_importances_, columns=['importance'])

,importance
Pclass,14.211322
Sex,53.317646
Age,8.503256
SibSp,4.481401
Parch,4.193231
Fare,6.223286
Embarked,3.067888
Deck,6.001970


In [25]:
test = prepare_dataset(test)

X_test = test.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

In [26]:
X_test.dtypes.to_frame('dtype').reset_index()

,index,dtype
0,Pclass,int64
1,Sex,object
2,Age,float64
3,SibSp,int64
4,Parch,int64
5,Fare,float64
6,Embarked,object
7,Deck,object


In [27]:
y_pred = model.predict(Pool(X_test, cat_features=[1, 6, 7]))

In [28]:
submission = pd.DataFrame(data=y_pred.astype('int64'), index=test['PassengerId'], columns=['Survived']).reset_index()

In [29]:
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [30]:
submission.to_csv('titanic/submission.csv', index=False)

`| 5806 (из 10341) | rklepov | 0.77511 |`

_Не такой плохой результат, я считаю ;)_